In [1]:
import pandas as pd
import numpy as np

In [6]:
all_models_0 = pd.read_csv('social_bias_scores.csv', index_col=0)
all_models_dataframes = [all_models_0]

for threshold in ['1', '2-5', '5', '7-5', '10']:
    
    dataframe_variable_name = 'all_models_' + "_".join(threshold.split('-'))
    file_name = 'social_bias_scores_threshold' + threshold + '.csv'
    vars()[dataframe_variable_name] = pd.read_csv(file_name, index_col=0)
    all_models_dataframes.append(vars()[dataframe_variable_name])

In [67]:
threshold_model_names = ['social_bias_scores_threshold1.csv',
                        'social_bias_scores_threshold2-5.csv',
                        'social_bias_scores_threshold5.csv',
                        'social_bias_scores_threshold7-5.csv',
                        'social_bias_scores_threshold10.csv']

In [44]:
model_names = [
    'bert-base-cased',
    'bert-base-uncased',
    'bert-large-uncased',
    'bert-large-cased',
    'bert-base-multilingual-uncased',
    'bert-base-multilingual-cased',
    'allenai/scibert_scivocab_uncased',
    'emilyalsentzer/Bio_ClinicalBERT',
    'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract',
    'nlpaueb/legal-bert-base-uncased',
    'GroNLP/hateBERT',
    'anferico/bert-for-patents',
    'jackaduma/SecBERT',
    'albert-base-v1',
    'albert-base-v2',
    'roberta-base',
    'distilroberta-base',
    'roberta-large',
    'huggingface/CodeBERTa-small-v1',
    'climatebert/distilroberta-base-climate-f',
    'xlm-roberta-base', 
    'distilbert-base-multilingual-cased']

In [45]:
bias_types = ['race-color',
             'gender', 
             'socioeconomic',
             'nationality', 
             'religion', 
             'age',
             'sexual-orientation',
             'physical-appearance',
             'disability']

bias_overall_weights = [516, 262, 172, 
               159, 105, 87, 
               84, 63, 60]

bias_stereo_weights = [473, 159, 157, 
                      148, 99, 73, 
                      72, 52, 57] # 1290

bias_antistereo_weights = [43, 103, 15, 
                          11, 6, 14, 
                          12, 11, 3] # 218

In [64]:
overall_metrics_dict = {'model': [],
                  'bias_type': [],
                  'metric_score': [],
                  'stereotype_score': [],
                  'antistereotype_score': []}

In [65]:
for model_name in model_names:
    
    overall_metric = 0
    overall_stereo = 0
    overall_antistereo = 0
    
    model = all_models_0[all_models_0['model'] == model_name]
    
    for i in range(0,9):
        
        bias_type = bias_types[i]
        bias_overall_weight = bias_overall_weights[i]
        bias_stereo_weight = bias_stereo_weights[i]
        bias_antistereo_weight = bias_antistereo_weights[i]
        metric_score = float(model[model['bias_type']==bias_type]['metric_score'])
        stereotype_score = float(model[model['bias_type']==bias_type]['stereotype_score'])
        antistereotype_score = float(model[model['bias_type']==bias_type]['antistereotype_score'])
        
        overall_metrics_dict['model'].append(model_name)
        overall_metrics_dict['bias_type'].append(bias_type)
        overall_metrics_dict['metric_score'].append(metric_score)
        overall_metrics_dict['stereotype_score'].append(stereotype_score)
        overall_metrics_dict['antistereotype_score'].append(antistereotype_score)
        
        overall_metric += (metric_score * bias_overall_weight)
        overall_stereo += (stereotype_score * bias_stereo_weight)
        overall_antistereo += (antistereotype_score * bias_antistereo_weight)
    
    overall_metrics_dict['model'].append(model_name)
    overall_metrics_dict['bias_type'].append('overall')
    overall_metrics_dict['metric_score'].append(round(overall_metric / sum(bias_overall_weights), 2))
    overall_metrics_dict['stereotype_score'].append(round(overall_stereo / sum(bias_stereo_weights), 2))
    overall_metrics_dict['antistereotype_score'].append(round(overall_antistereo / sum(bias_antistereo_weights), 2))

In [66]:
output = pd.DataFrame(overall_metrics_dict)
output.to_csv('social_bias_scores.csv')

In [68]:
for threshold_model_name in threshold_model_names:
    
    threshold_model = pd.read_csv(threshold_model_name, index_col=0)
    
    overall_metrics_dict = {'model': [],
                            'bias_type': [],
                            'metric_score': [],
                            'stereotype_score': [],
                            'antistereotype_score': [],
                            'neutral_score': []}

    for model_name in model_names:

        overall_metric = 0
        overall_stereo = 0
        overall_antistereo = 0
        overall_neutral = 0

        model = threshold_model[threshold_model['model'] == model_name]

        for i in range(0,9):

            bias_type = bias_types[i]
            bias_overall_weight = bias_overall_weights[i]
            bias_stereo_weight = bias_stereo_weights[i]
            bias_antistereo_weight = bias_antistereo_weights[i]
            metric_score = float(model[model['bias_type']==bias_type]['metric_score'])
            stereotype_score = float(model[model['bias_type']==bias_type]['stereotype_score'])
            antistereotype_score = float(model[model['bias_type']==bias_type]['antistereotype_score'])
            neutral_score = float(model[model['bias_type']==bias_type]['neutral_score'])

            overall_metrics_dict['model'].append(model_name)
            overall_metrics_dict['bias_type'].append(bias_type)
            overall_metrics_dict['metric_score'].append(metric_score)
            overall_metrics_dict['stereotype_score'].append(stereotype_score)
            overall_metrics_dict['antistereotype_score'].append(antistereotype_score)
            overall_metrics_dict['neutral_score'].append(neutral_score)

            overall_metric += (metric_score * bias_overall_weight)
            overall_stereo += (stereotype_score * bias_stereo_weight)
            overall_antistereo += (antistereotype_score * bias_antistereo_weight)
            overall_neutral += (neutral_score * bias_overall_weight)

        overall_metrics_dict['model'].append(model_name)
        overall_metrics_dict['bias_type'].append('overall')
        overall_metrics_dict['metric_score'].append(round(overall_metric / sum(bias_overall_weights), 2))
        overall_metrics_dict['stereotype_score'].append(round(overall_stereo / sum(bias_stereo_weights), 2))
        overall_metrics_dict['antistereotype_score'].append(round(overall_antistereo / sum(bias_antistereo_weights), 2))
        overall_metrics_dict['neutral_score'].append(round(overall_metric / sum(bias_overall_weights), 2))
    
    output = pd.DataFrame(overall_metrics_dict)
    output.to_csv(threshold_model_name)